<a href="https://colab.research.google.com/github/Thuffy13/CNNs-For-DNA-Sequence-Analysis/blob/main/Superloop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os, h5py
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow import keras
import model_zoo
import tfomics
import tensorflow as tf

filepath = 'truncated_deepsea_dataset.h5'
dataset = h5py.File(filepath, 'r')

with h5py.File(filepath, 'r') as dataset:
    x_train = np.array(dataset['X_train']).astype(np.float32)
    y_train = np.array(dataset['Y_train']).astype(np.float32)
    x_valid = np.array(dataset['X_valid']).astype(np.float32)
    y_valid = np.array(dataset['Y_valid']).astype(np.int32)
    x_test = np.array(dataset['X_test']).astype(np.float32)
    y_test = np.array(dataset['Y_test']).astype(np.int32)

x_train = np.squeeze(x_train).transpose([0,2,1])
x_valid = np.squeeze(x_valid).transpose([0,2,1])
x_test = np.squeeze(x_test).transpose([0,2,1])

N, L, A = x_train.shape
num_labels = y_valid.shape[1]
print(x_train.shape)

pip install -U tensorflow-addons

from tensorflow.python.tools import module_util
import tensorflow as tf
import tensorflow_addons as tfa

'''
bn_before = True
bn_after = False
ln_before = False
ln_after = False
in_before = False
in_after = False
first_layer_pass = True
'''

keras.backend.clear_session() 

#input-shape = (L,A)
def cnn4(input_shape, output_shape, multiplier, bn_before, bn_after, ln_before, ln_after, in_before, in_after, first_layer_pass):
  # l2 regularization
  l2 = keras.regularizers.l2(1e-6)

  multiplier = 2;

  # input layer
  inputs = keras.layers.Input(shape=input_shape)

  # layer 1 - convolution
  nn = keras.layers.Conv1D(filters=32*multiplier,
                          kernel_size=19,
                          strides=1,
                          activation=None,
                          use_bias=False,
                          padding='same',
                          kernel_regularizer=l2, 
                          )(inputs)
  if (bn_before == True) and (first_layer_pass == False):        
    nn = keras.layers.BatchNormalization()(nn)
  elif ln_before == True:
    nn = keras.layers.LayerNormalization()(nn)
  elif in_before == True:
    nn = tfa.layers.InstanceNormalization()(nn)

  nn = keras.layers.Activation('relu')(nn)

  if bn_after == True:        
    nn = keras.layers.BatchNormalization()(nn)
  elif ln_after == True:
    nn = keras.layers.LayerNormalization()(nn)
  elif in_after == True:
    nn = tfa.layers.InstanceNormalization()(nn)
  nn = keras.layers.MaxPool1D(pool_size=4)(nn)
  nn = keras.layers.Dropout(0.1)(nn)

  # layer 2 - convolution
  nn = keras.layers.Conv1D(filters=128*multiplier,
                          kernel_size=7,
                          strides=1,
                          activation=None,
                          use_bias=False,
                          padding='same',
                          kernel_regularizer=l2, 
                          )(nn)        
  if bn_before == True:        
    nn = keras.layers.BatchNormalization()(nn)
  elif ln_before == True:
    nn = keras.layers.LayerNormalization()(nn)
  elif in_before == True:
    nn = tfa.layers.InstanceNormalization()(nn)

  nn = keras.layers.Activation('relu')(nn)

  if bn_after == True:        
    nn = keras.layers.BatchNormalization()(nn)
  elif ln_after == True:
    nn = keras.layers.LayerNormalization()(nn)
  elif in_after == True:
    nn = tfa.layers.InstanceNormalization()(nn)
  nn = keras.layers.MaxPool1D(pool_size=25)(nn)
  nn = keras.layers.Dropout(0.1)(nn)

  # layer 3 - Fully-connected 
  nn = keras.layers.Flatten()(nn)
  nn = keras.layers.Dense(512*multiplier,
                          activation=None,
                          use_bias=False,
                          kernel_regularizer=l2, 
                          )(nn)      
  if bn_before == True:        
    nn = keras.layers.BatchNormalization()(nn)
  elif ln_before == True:
    nn = keras.layers.LayerNormalization()(nn)
  elif in_before == True:
    nn = tfa.layers.InstanceNormalization()(nn)

  nn = keras.layers.Activation('relu')(nn)

  if bn_after == True:        
    nn = keras.layers.BatchNormalization()(nn)
  elif ln_after == True:
    nn = keras.layers.LayerNormalization()(nn)
  elif in_after == True:
    nn = tfa.layers.InstanceNormalization()(nn)
  nn = keras.layers.Dropout(0.5)(nn)

  # Output layer
  logits = keras.layers.Dense(output_shape, activation='linear', use_bias=True,  
                                  kernel_initializer='glorot_normal',
                                  bias_initializer='zeros')(nn)
  outputs = keras.layers.Activation('sigmoid')(logits)

  # create keras model
  return keras.Model(inputs=inputs, outputs=outputs)
'''
modely = cnn4((L,A), 12, 2, True,
False,
False,
False,
False,
False,
False)

modely.summary()
'''
# save model weights
#model.save_weights('my_model_weights.h5')

'''
# set up optimizer and metrics
auroc = keras.metrics.AUC(curve='ROC', name='auroc')
aupr = keras.metrics.AUC(curve='PR', name='aupr')
optimizer = keras.optimizers.Adam(learning_rate=0.001)
loss = keras.losses.BinaryCrossentropy(from_logits=False, label_smoothing=0.0)
modely.compile(optimizer=optimizer,
                loss=loss,
                metrics=[auroc, aupr])

'''


import pickle
from tfomics import moana, impress
from pandas import DataFrame
from tfomics import evaluate
from googleapiclient.http import MediaFileUpload
from google.colab import auth
from googleapiclient.discovery import build

norms = [[True, False, False, False, False, False, True], [True, False, False, False, False, False, False],
         [False, True, False, False, False, False, True], [False, True, False, False, False, False, False],
         [False, False, True, False, False, False, True], [False, False, True, False, False, False, False],
         [False, False, False, True, False, False, True], [False, False, False, True, False, False, False],
         [False, False, False, False, True, False, True], [False, False, False, False, True, False, False],
         [False, False, False, False, False, True, True], [False, False, False, False, False, True, False]]

#You could also replicate the above list by making two matrices which are the 6x6 identity matrixa concatenated with a 7th column which is
#1 for first_layer_pass = True and 0 for first_layer_pass = False and check the conditions of 1 or 0 instead of True or False

norm_iter = 0

for norm in norms:
  norm_iter += 1
  q_val_list = []
  ground_truth_hits = []
  any_motif_hits = []
  mean_auroc = []
  mean_aupr = []
  aurocy = []
  aupry = []
  #Change to i in range(5) for the full training loop; in range(1) is just to expedite the testing process
  for i in range(5):
    modely = cnn4((L,A), 12, 2, norm[0], norm[1], norm[2], norm[3], norm[4], norm[5], norm[6])
    
    auroc = keras.metrics.AUC(curve='ROC', name='auroc')
    aupr = keras.metrics.AUC(curve='PR', name='aupr')
    optimizer = keras.optimizers.Adam(learning_rate=0.001)
    loss = keras.losses.BinaryCrossentropy(from_logits=False, label_smoothing=0.0)
    modely.compile(optimizer=optimizer,
                    loss=loss,
                    metrics=[auroc, aupr])

    keras.backend.clear_session() 
    
    #Train the model
    # early stopping callback
    es_callback = keras.callbacks.EarlyStopping(monitor='val_aupr', #'val_aupr',#
                                                patience=12, 
                                                verbose=1, 
                                                mode='max', 
                                                restore_best_weights=False)
    # reduce learning rate callback
    reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_aupr', 
                                                    factor=0.2,
                                                    patience=4, 
                                                    min_lr=1e-7,
                                                    mode='max',
                                                    verbose=1) 

    # train model
    history = modely.fit(x_train, y_train, 
                        epochs=100,
                        batch_size=100, 
                        shuffle=True,
                        validation_data=(x_valid, y_valid), 
                        callbacks=[es_callback, reduce_lr])
    

    # evaluate model on test set
    results = modely.evaluate(x_test, y_test, batch_size=512)  

    #auroc = []
    #aupr = []
    aurocy.append(results[1])
    aupry.append(results[2])

    if i!= 4:
      mean_auroc.append(0)
      mean_aupr.append(0)
    elif i==4:
      meany_auroc = 0
      for i in aurocy:
        meany_auroc += i

      meany_auroc = meany_auroc/len(aurocy)

      meany_aupr = 0
      for i in aupry:
        meany_aupr += i

      meany_aupr = meany_aupr/len(aupry)


      mean_auroc.append(meany_auroc)
      mean_aupr.append(meany_aupr)
    #aupr = [results[2]]

    

    #Create a  pandas dataframe to store our metrics in
    #df = DataFrame({'auroc': auroc, 'aupr': aupr})
    #df

    #Write this dataframe to either a csv or excel file
    #Note that this does not work at the moment and I am unsre why
    #%%file 'NormData.xlsx'
    #!type df.to_excel('NormData.xlsx', sheet_name='sheet1', index=False)
    #df.to_csv('CNN4_Norm_Results.csv', index=False)


    #W4 is the position prbability matrix
    W4 = moana.filter_activations(x_test, modely, layer=2, window=20, threshold=0.5)
    #print('W4', W4)

    fig = plt.figure(figsize=(20,4))
    W_df = impress.plot_filters(W4, fig, num_cols=8, alphabet='ACGT')


    # clip filters
    W_clipped = moana.clip_filters(W4, threshold=0.5, pad=3)

    #Need to find a way to find the path to my google drive to save each meme file for each iteration of the model to a different filename in google drive
    #i.e: cnn4_meme_iter1, cnn4_meme_iter2, cnn4_meme_iter3, cnn4_meme_iter4, cnn4_meme_iter5
    #pathtogoogledrive = os.path.join()

    # generate meme file
    meme_name = 'cnn4_filters_+' + 'norm_iter(' + str(norm_iter) + ')_' + 'training_iter(' + str(i) + ').meme'
    moana.meme_generate(W_clipped, output_file= meme_name, prefix='filter')

    #from google.colab import auth
    auth.authenticate_user()
    #from googleapiclient.discovery import build
    drive_service = build('drive', 'v3')

    with open('/tmp/' + meme_name, 'a') as f:
      f.write('')

    print('/tmp/' + meme_name + ' contains:')
    !cat /tmp/to_upload.meme

    #from googleapiclient.http import MediaFileUpload

    file_metadata = {
      'name': meme_name,
      'mimeType': 'text/plain'
    }
    media = MediaFileUpload(meme_name, 
                            mimetype='text/plain',
                            resumable=True)
    created = drive_service.files().create(body=file_metadata,
                                          media_body=media,
                                          fields='id').execute()
    print('File ID: {}'.format(created.get('id')))

    '''
    #Now we need to upload the weights of our model to the google drive
    
    weights = []

    #auth.authenticate_user()
  
    #drive_service = build('drive', 'v3')


    weights_name = 'cnn4_filters_bnbefore_fptrue_weights' + str(i) + '.h5'
    weights_name = 'cnn4_filters_' + str(norm_iter) + str(i) + '.h5'

    for layer in modely.layers:
      weights.append(layer.get_weights()) # list of numpy arrays\

    outfile = open(weights_name, 'wb')

    pickle.dump(weights, outfile)

    outfile.close()

 
    with open('/tmp/' + weights_name, 'a') as f:
      f.write('')

    print('/tmp/' + weights_name + ' contains:')
    !cat /tmp/to_upload.csv


    file_metadatay = {
      'name': weights_name,
      'mimeType': 'application/vnd.google-apps.file'
    }

    

    mediay = MediaFileUpload(weights_name,
                            resumable=True
  )
    createdy = drive_service.files().create(body=file_metadatay,
                                          media_body=mediay,
                                          fields='id').execute()
    print('File ID: {}'.format(createdy.get('id')))
    '''
    weights_name = 'cnn4_filters_' + 'norm_iter(' + str(norm_iter) + ')_' + 'training_iter(' + str(i) + ').h5'
    # Save the entire model as a SavedModel.
    #!mkdir -p saved_model
    #modely.save(weights_name) 

    #!ls saved_model

    # Contains an assets folder, saved_model.pb, and variables folder.
    #!ls saved_model/my_model

    outfile = open(weights_name, 'wb')

    pickle.dump(modely.save_weights(weights_name), outfile)

    outfile.close()
    
    file_metadatay = {
        'name': weights_name
      }

      

    mediay = MediaFileUpload(weights_name,
                              resumable=True
    )
    createdy = drive_service.files().create(body=file_metadatay,
                                            media_body=mediay,
                                            fields='id').execute()
    print('File ID: {}'.format(createdy.get('id')))


    #Perform ground truth analysis
    motif_path = meme_name
    output_path = 'cnn4_filters'
    jaspar_path = 'JASPAR_CORE_2016_vertebrates.meme'
    stdout, stderr = moana.tomtom(motif_path, jaspar_path, output_path, evalue=True, thresh=0.5, dist='pearson', png=None, tomtom_path='./src/tomtom')


    num_filters = moana.count_meme_entries(motif_path)


    #CNN-4 results
    file_path = 'cnn4_filters/tomtom.tsv'
    match_fraction4, match_any4, filter_match4, filter_qvalue4, min_qvalue4, num_counts4 \
            = evaluate.motif_comparison_synthetic_dataset(file_path, num_filters=num_filters)
    #print("CNN-4 analysis")
    #print("Fraction of hits to ground truth motifs: %.3f"%(match_fraction4))
    #print("Fraction of hits to any motif: %.3f"%(match_any4))
    #print('Hits to ground truth motif for each filter:')
    #print(filter_match4)
    #print('min q value: ', min_qvalue4)

    q_val_list.append(min_qvalue4)
    ground_truth_hits.append(match_fraction4)
    any_motif_hits.append(match_any4)




  #Calculate the mean aupr and auroc
  #mean_auroc = []
  #mean_aupr = []
  '''

  meany_auroc = 0
  for i in aurocy:
    meany_auroc += i

  meany_auroc = meany_auroc/len(aurocy)

  meany_aupr = 0
  for i in aupry:
    meany_aupr += i

  meany_aupr = meany_aupr/len(aupry)


  mean_auroc.append(meany_auroc)
  mean_aupr.append(meany_aupr)
  '''

  panda_name = 'CNN4_Norm_Results' + 'norm_iter(' + str(norm_iter) + ')_' + 'training_iter(' + str(i) + '.csv'

  df = pd.DataFrame(list(zip(aurocy, aupry, mean_auroc, mean_aupr, any_motif_hits, ground_truth_hits, q_val_list)),
               columns =['auroc', 'aupr', 'mean_auroc', 'mean_aupr', 'any motif hits', 'ground truth hits', 'min q vals'])
  
  df.to_csv(panda_name, index=False)
  
  with open('/tmp/' + panda_name, 'a') as f:
      f.write('')

  print('/tmp/' + panda_name + ' contains:')
  !cat /tmp/to_upload.csv

  file_metadata = {
    'name': panda_name
  }
  media = MediaFileUpload(panda_name, 
                          resumable=True)
  
  created = drive_service.files().create(body=file_metadata,
                                        media_body=media,
                                        fields='id').execute()
  print('File ID: {}'.format(created.get('id')))



  #df


